In [1]:
def getInput(day):
   f = open("day" + str(day) + "_22" + ".input", "r")
   return f.readlines()

In [2]:
# day 1
import numpy as np

# Splits an array of strings (representing lines of an file in our case)
# into an array of two arrays of strings, the first containing all strings/lines 
# before the first blank line ('\n'), while the last contains those after the
# blank line. The blank line itself is not part of any of these two.
# If the input array of strings contains no blank line at all, this function 
# returns an array containing the inputs array of strings.  
def splitOnFirstBlankLine(stringArray):
    if '\n' in stringArray:
        return [stringArray[:stringArray.index('\n')], stringArray[stringArray.index('\n')+1:]]
    else:
        return [stringArray]

# splits an arrays of strings (representing lines of an file in our case) 
# into an array of arrays of strings, each containing the strings between to
# blank lines ('\n')
# For example the file
# A
# B
#
# C
#
# D
# E
# would be split up into [['A', 'B'], ['C'], ['D', 'E']]
def splitOnBlankLines(stringArray, result=[]):
    splits = splitOnFirstBlankLine(stringArray)
    result.append(splits[0])
    if len(splits) == 1:
        return result
    else: # len == 2
        return splitOnBlankLines(splits[1], result)

# Makes an array of arrays of Integers from an input array of arrays of strings.
# Input array must not contain anything else than an array of arrays of strings,
# otherwise an exception will be thrown
# For example, [['2', '3'], ['5'], ['7', '11']] will be converted into
# [[2, 3], [5], [7, 11]]
def intify(arrayOfStringArrays):
    arrayOfIntArrays = []
    for stringArray in arrayOfStringArrays:
        intArray = [int(string) for string in stringArray]
        arrayOfIntArrays.append(intArray)
    return arrayOfIntArrays

def solveDay1(task):
    # read input
    inputLines = getInput(1)
    
    # split up
    splittedIntoChunks = splitOnBlankLines(inputLines)
    
    # make integers
    intChunks = intify(splittedIntoChunks)
    
    # make an array of sums of chunks
    sums = [np.sum(split) for split in intChunks]
    # and sort by size
    sums.sort()
    
    if task == 1: 
        # the top elve
        return sums[-1]
    if task == 2:
        # three top elves summed together
        return np.sum(sums[-3:])
    
print(solveDay1(1)) 
print(solveDay1(2)) 


72511
214836


In [3]:
#day 2
import numpy as np

# the first half of scoresByShape is used by both tasks (intpretation of A, B, C):
# A for Rock, B for Paper, and C for Scissors.
# the second half of scoresByShape is only needed for the first task (wrong interpretation of X, Y, Z):
# X for Rock, Y for Paper, and Z for Scissors.
scoresByShape = {'A':1, 'B':2, 'C':3, 'X':1, 'Y':2, 'Z':3}

# only for the second task (real interpretation of X, Y, Z)
# X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win.
neededScores = {'X':0, 'Y':3, 'Z':6}

def cantorPairing(x,y):
    return (pow(x,2) + x + 2 * x * y + 3 * y + pow(y,2))/2

# the points that we (with nyShape) receive against them (with theirShape)
def outcomeScore(theirShape, myShape):
    ts = scoresByShape[theirShape]
    ms = scoresByShape[myShape.rstrip()]
    # ugly ternary expression - can replaced with some math involving cantor and interpolation polynomial see below
    # return 3 if ts == ms else (6 if (ms == 1 and ts == 3) else (0 if (ms == 3 and ts == 1) else (0 if ts > ms else 6)))
    x = cantorPairing(ts, ms)
    return int(round(331 * pow(x,8)/304944640 - 921467 * pow(x,7)/6861254400 + 16026271 * pow(x,6)/2287084800 - 63041351 * pow(x,5)/311875200 + 489434749 * pow(x,4)/138611200 - 261604767803 * pow(x,3)/6861254400 + 8799384401 * pow(x,2)/35735700 - 3137692909 * x/3665200 + 421473/350))

# the points we score in a duell, represented by a string defined by 
# '<THEIR_SHAPE><SINGLE_SPACE><THEIR_SHAPE><OPTIONAL_NEWLINE>', 
# where every shape is one of  A, B, C, X, Y, Z. Examples: 
# - 'C Y'
# - 'A Z\n'
# - 'B Z'
# - 'A C'
# - 'X Y\n'
#  or similar
def score(duell):
    shapes = duell.split(' ')
    return outcomeScore(shapes[0], shapes[1]) + scoresByShape[shapes[1].rstrip()]

# the duell given by a strategy, represented by a string defined by 
# '<THEIR_SHAPE><SINGLE_SPACE><STRATEGIC_OUTCOME><OPTIONAL_NEWLINE>',
# where every shape is one of  A, B, C and 
# each strategic outcome one of X, Y, Z. Examples:
# - 'A Y'
# - 'A Z\n'
# - 'B Z'
#  or similar
def duellFor(strategy):
    strategyParts = strategy.split(' ')
    theirShape = strategyParts[0]
    neededScore = neededScores[strategyParts[1].rstrip()]
    for shape in ['A', 'B', 'C']:
        if outcomeScore(theirShape, shape) == neededScore:
            return theirShape + ' ' + shape
    return 'oops' # should not come here

def solveDay2(task):
    # determine duells
    if task == 1:
         duells = getInput(2)
    if task == 2:
        strategies = getInput(2)
        duells = [duellFor(strategy) for strategy in strategies]
    
    # calculate all scores
    scores = [score(duell) for duell in duells]

    # sum up scores and return
    return np.sum(scores)
    
print(solveDay2(1))
print(solveDay2(2))

15691
12989


In [4]:
# day 3
import numpy as np

def priority(itemType):
    corr =  96 if itemType.lower() == itemType else 38
    return ord(itemType) - corr

def inAllCollections(itemType, collections):
    for collection in collections:
        if not itemType in collection:
            return False
    return True

def itemTypeInAllCollections(collections):
    for itemType in collections[0]:
        if inAllCollections(itemType, collections):
            return itemType

def itemTypeInBothCompartments(rucksack):
    compartment1 = rucksack[:int(len(rucksack)/2)]
    compartment2 = rucksack[int(len(rucksack)/2):]
    return itemTypeInAllCollections([compartment1, compartment2])

def solveDay3(task):
    rucksacks = getInput(3)
    if task == 1:
        wrongItemTypes = [itemTypeInBothCompartments(rucksack) for rucksack in rucksacks]
        wrongPriorities = [priority(itemType) for itemType in wrongItemTypes]
        return np.sum(wrongPriorities)
    if task == 2:
        groups = np.array_split(rucksacks, int(len(rucksacks)/3))
        groupItemTypes = [itemTypeInAllCollections(group) for group in groups]
        groupPriorities = [priority(itemType) for itemType in groupItemTypes]
        return np.sum (groupPriorities)
    
print(solveDay3(1))
print(solveDay3(2))

8153
2342


In [5]:
# day 4

def oneIncludesOther(pair):
    both = pair.split(',')
    first = [int(x) for x in both[0].split('-')]
    second= [int(x) for x in both[1].split('-')]
    return first[0] <= second[0] and first[1] >= second[1] or first[0] >= second[0] and first[1] <= second[1]

def overlaps(pair):
    both = pair.split(',')
    first = [int(x) for x in both[0].split('-')]
    second= [int(x) for x in both[1].split('-')]
    return first[1] >= second[0] and first[0] <= second[1] or first[0] <= second[1] and first[1] >= second[0]

def solveDay4(task):
    pairs = getInput(4)
    if task == 1:
        result = 0
        for pair in pairs:
            if oneIncludesOther(pair):
                result += 1
        return result
    if task == 2:
        result = 0
        for pair in pairs:
            if overlaps(pair):
                result += 1
        return result

print(solveDay4(1))
print(solveDay4(2))

599
928


In [6]:
# day 5
def stacksFrom(data):
    rows = []
    for line in data:
        if '[' in line:
            line = line.rstrip()
            rows.append(line.split(' '))
    #transpose
    stacks = [[row[i] for row in rows] for i in range(len(rows[0]))]
    #clean
    for stack in stacks:
        while '[0]'in stack:
            stack.remove('[0]')
    return stacks

def movesFrom(data):
    moves = []
    for line in data:
        if 'move' in line:
            line = line.rstrip()
            moves.append(list(map( lambda s: int(s), list(filter(lambda s: s.isdigit(), line.split(' '))))))
    return moves

def stacksAfterMove(stacks, move, task):
    sourceIdx = move[1]-1
    targetIdx = move[2]-1
    if task == 1:
        stacks[targetIdx] = stacks[sourceIdx][:move[0]][::-1] + stacks[targetIdx]
    else:
        stacks[targetIdx] = stacks[sourceIdx][:move[0]] + stacks[targetIdx]
    stacks[sourceIdx] = stacks[sourceIdx][move[0]:]
    return stacks

def solveDay4(task):
    data = getInput(5)
    stacks = stacksFrom(data)
    moves = movesFrom(data)
    for move in moves:
        stacks = stacksAfterMove(stacks, move, task)
    message = ''
    for stack in stacks:
        message = message + stack[0][1]
    return message

print(solveDay4(1))
print(solveDay4(2))

TLNGFGMFN
FGLQJCMBD


In [8]:
# day 6

def numberProcessedBeforeReachingDistinctOfLenghth(numberOfDistinct):
    data = list(getInput(6)[0])
    for i in range(len(data) - numberOfDistinct):
        chunk = data[i:i+numberOfDistinct]
        if len(set(chunk)) == numberOfDistinct:
            return i + numberOfDistinct


def solveDay6(task):
    if task == 1:
        return numberProcessedBeforeReachingDistinctOfLenghth(4)
    if task == 2:
        return numberProcessedBeforeReachingDistinctOfLenghth(14)

print(solveDay6(1))
print(solveDay6(2))

1658
2260


In [21]:
from collections import namedtuple
import numpy as np

File = namedtuple('File', ['name', 'size'])
Dir = namedtuple('Dir', ['name', 'files', 'subdirs'])
fileA = File('A', 5)
fileB = File('B', 7)
fileC = File('C', 11)
fileD = File('D', 17)
subsubdir11 =  Dir('subsubdir11',[fileC, fileA], [])
subdir1 = Dir('subdir1',[fileA, fileB], [subsubdir11])
subdir2 = Dir('subdir2',[fileC], [])
dir = Dir('dir', [], [subdir1, subdir2])

def sizeOf(dir):
    size = np.sum([file.size for file in dir.files])
    for subdir in dir.subdirs:
        size = size + sizeOf(subdir)
    return size

def subdirWithName(dirs, name):
    for dir in dirs:
        for subdir in dir.subdirs:
            if subdir.name == name:
                return subdir
    for dir in dirs:
        return subdirWithName(dir.subdirs, name)

print(sizeOf(dir))
    

subsubdir11 16
subdir1 28
subdir2 11
dir 39.0
39.0


In [10]:
# day 7

def buildDirTree(harddisk):
    for line in harddisk:
        if line

def solveDay7(task):
    if task == 1:
        harddisk = getInput(7)
        return 1
    if task == 2:
        return 2

print(solveDay7(1))
print(solveDay7(2))

1
2
